
## (p) 1.4 机器学习基础(Machine Learning Basics)

本章介绍本书其余部分涉及到的机器学习的最重要原理. 

> 对于新手, 推荐阅读 **Murphy (2012)** or **Bishop(2006)**

大部分机器学习算法需要设置 **超参数(hyperparameters, 必须在算法外设定)**, 机器学习是一种应用统计学, 用计算机来统计地估计复杂函数而不太关注这些函数提供的置信空间(confidence intervals), 并且有两个核心方法来统计: 频率派估计(frequentist estimator)和贝叶斯推断(Bayesian inference). 大部分机器学习算法可以划分为两类: **监督学习(supervised learning)** 和 **无监督学习(unsupervised learning)** . 大部分深度学习算法都是基于一个叫做 **随机梯度下降(stochastic gradient descent)** 的优化算法. 我们将介绍如何如何将各种算法组件(优化算法, 代价航函数(cost function), 模型, 数据集(dataset))来构建机器学习算法.

### (p) 1.4.1 学习算法

 Mitchell (1997) 提供了一个简洁的学习算法的定义: **对于某任务 T 和性能度量 P , 一个计算机程序能够从经验 E 中学习是指, 通过经验 E 改进后, 它在任务 T 上由性能度量 P 衡量的性能有所提升.** E, T, P 的种类相当广泛, 所以我们不会形式化得定义它们, 我们给出一些直观的描述和例子:
 
 **任务 T**
 
 学习过程本身不是任务, 学习是实现任务的手段. 例如, 要让机器人行走, 行走就是任务. 机器学习任务定义为机器学习系统如何处理 **样本(example, 样本就是一些特征(features)的集合, 通常表示为向量 $x \in \mathbb{R}^n$ 表示 $n$ 个特征 )** . 能用机器学习解决的任务主要有以下几类:
 
 * **分类(Classfication)** : 将输入分类为 $k$ 类(有可能还带有概率), $f: \mathbb{R}^n \rightarrow \{ 1, \ldots, k\}, y = f(\boldsymbol{x})$, 返回一个数值码(numeric code) $y$ . 例如物体识别(object recognition), 人脸识别.
 * **Classiﬁcation with missing inputs**: 如果输入中有些内容被缺失, 我们的学习算法就需要一组函数来处理有不同缺失情况的子集. 我们可以通过学习所有相关变量的概率分布, 然后通过边缘化( marginalizing)缺失变量来解决分类任务.$n$ 个输入变量需要 $2^n$ 个不同的分类函数来应对每种可能的缺失情况, 但是计算机程序只需要学习一个描述联合概率分布的函数.
 * **回归(Regression)**: 对给定输入输出预测数值. $f: \mathbb{R}^n \rightarrow \mathbb{R}$. 例如预测证劵未来价格, 预测投保人的索赔金额.
 * **转录(Transcription)**: 将非结构化的数据转化成离散文本形式, 例如光学字符识别和语音识别.
 * **机器翻译(machine translation)**
 * **结构化输出(structured output)** : 输出为各个元素有关联的向量. 结构化输出与许多机器学习任务(例如上面提到的机器翻译和转录)相关联, 例如将自然语言按照语法解析为为语句树. 还有一个例子就是像素级分割.
 * **异常检测(Anomaly detection)** : 从一堆事件或物体中筛选出异常或非典型的. 例如信用卡欺诈识别.
 * **合成和采样(Sythesis and sampling)** : 用机器学习来生成和训练样本类似的新样本. 例如游戏中大型物体或风景的纹理, 以及语音合成.
 * **缺失值填补(Imputation of missing values)** : 算法为输入中确实的条目预测值. 
 * **去噪(denoising)** : 算法的输入是 **干净样本(clean example)** $\boldsymbol{x} \in \mathbb{R}^n$ 通过未知损坏过程后得到的 **损坏样本(corruption example)** $\tilde{\boldsymbol{x}} \in \mathbb{R}^n$ , 算法通过损坏样本预测干净样本, 或者更一般的预测条件概率分布 $p(\boldsymbol{x} | \boldsymbol{\tilde{\boldsymbol{x}}})$ .
* **密度估计(density estimation)** 或 **概率质量函数估计(probability mass function estimation)** : 算法学习函数 $p_{\text{model}}(\boldsymbol{x}): \mathbb{R}^n \rightarrow \mathbb{R}$, 其中 $p$ 可以解释为样本采样空间的概率密度函数(如果是连续型)或概率质量函数(如果是离散型). 上面提到的任务大多只需要至少能隐式地捕获(capture)概率分布的结构, 而密度估计必须显式地捕获该分布. 例如, 我们可以将密度估计的结果用于缺失值填补任务. (但实际情况是密度估计不能解决所有这类问题).

**性能度量(Performance Measure) P**

对于分类, 缺失输入的分类和转录, *P* 往往是衡量模型的 **准确率(accuracy, 模型产生正确输出的比率, 或者用错误率(error rate)来度量)** . 我们通常把错误率称为 **0-1损失的期望** , 在一定特定的样本上, 如果结果是对的, 那么 0-1 损失为0, 否则是1. 但是 0 - 1 损失模型或者准确率错误率对于密度估计这类任务而言, 是没有意义的, 我们可以对模型的每个样本都输出一个连续数值的得分, 最常见的方法就是输出模型在一些样本上概率对数的平均值.  我们使用 **测试集(test set)** 来度量性能. 有时候, 很难确定到底要度量什么, 比如在度量转录任务时, 我们是应该度量整个序列的准确率还是更加细粒度(fine-grained)地对序列中正确的那部分元素以正面评价, 在度量回归任务时, 我们是应该很多的惩罚(penalize)那些频繁的中等错误还是少数几个很大的错误?	这些选取取决于应用. 又有些时候, 我们知道应该度量什么, 但是度量他们又不现实, 例如密度估计, 很多最好的概率模型只能隐式的表现其概率分布, 在这种情况, 我们必须设计一个对应于设计对象的替代标准, 或者设计一个理想标准的近似. 

**经验(experience) E**

按照在学习过程中算法被允许有什么类型的经验, 可以把机器学习算法分为 **无监督(unsupervised)** 和 **监督(supervised)学习**. 

本书中大部分学习算法可以为认为是在整个 **数据集(dataset, 很多样本组成的集合)** 上获取经验. 

最古老的用来给统计学家和机器学习研究人员学习的数据集是 Iris(鸢尾花卉) 数据集, 每个样本包括该植物不同部分的测量结果(萼片长度、萼片宽度之类的), 这个数据集中包含三个品种, 并且标明了每个样本的品种.

* 无监督学习从数据集中学习这个数据集上有用的结构性质. 在深度学习的背景下, 学习生成数据集的整个概率分布(例如密度统计着这种显式的, 或者合成, 去噪这些隐式的). 还有些其他类型的, 例如 **聚类(clustering)** , 将数据集中的所有样本按照相似的放在一起来分成若干个集群.
* 监督学习也是从包含很多特征的数据集中学习, 不过每个样本都有一个与之关联的 **标签(label)** 或 **目标(target)** . 例如, 监督学习通过研究 Iris 数据集, 学习如何通过测量信息来将样本划分为三个品种.

粗略得说, 无监督学习从一些随机向量 $\boldsymbol{\mathtt{x}}$ 中观察并隐式或显式的学习概率分布 $p(\boldsymbol{\mathtt{x}})$ , 或这个概率分布的有趣的性质, 而监督学习从一些随机向量$\boldsymbol{\mathtt{x}}$ 和与之关联的值或向量 $\boldsymbol{\mathtt{y}}$观察学习来预测 $\boldsymbol{\mathtt{y}}$ (也就是估计 $p(\boldsymbol{\mathtt{y}} | \boldsymbol{\mathtt{x}})$ ). 监督学习和无监督学习并非形式化定义的术语, 所以他们之间的界限往往也比较模糊, 我们知道条件概率的链式法则 $p(\boldsymbol{\mathtt{x}} \in \mathbb{R}^n) = \prod_{i = 1}^n p(x_i | x_1, \ldots, x_{i - 1})$, 所以我们可以把模型 $p(\boldsymbol{\mathtt{x}})$的无监督学习转化为 $n$ 个监督学习. 又由贝叶斯法则 $p(y | \boldsymbol{\mathtt{x}}) = \frac{p(\boldsymbol{\mathtt{x}}, y')} {\sum_{y'}p(\boldsymbol{\mathtt{x}}, y')}$ , 可以把监督学习转化为无监督学习. 

一般的, 回归, 分类, 结构化输出问题被称为监督学习, 密度估计被称为无监督学习. 还有其他一些变种学习范例, 例如 **半监督学习(semi-supervised)** , 一些样本包含监督目标而另外一些不包含. 而在 **多实例学习(multi-instance learning)** 中, 一个包含若干样本的集合(称为包(bag))只是被标记为包含还是没有包含某一类的样本, 而这个集合中的样本本身没有做任何标记. 有些学习算法并不是只从固定的数据集中学习, **强化学习(reinfocement learning)** 会和环境交互, 也就是说学习系统和它的训练过程之间还有反馈回路(feedback loop), 利用深度学习方法的强化学习称为 **深度强化学习(deep learning approach to reinfocement learning)** , 强化学习不在本书范围内. 

一种常见的表示数据集的方式是使用 **设计矩阵(design matrix)** , 一个设计矩阵中每一行代表一个样本, 而每一列则代表不同的特征. 但是, 设计矩阵中的每一个向量都必须是相同的大小, 这在某些情况下是不可能的, 例如如果有一堆不同高度宽度的照片的集合, 肯定不能用相同大小的向量来表示所有的照片, 我们之后会介绍怎么处理这样的 **异构数据(heterogeneous data)** , 并且在这种情况下我们使用集合表示 ${\boldsymbol{x}^{(1)}, \ldots, \boldsymbol{x}^{(m)}}$, 其中任意两个向量 $\boldsymbol{x}^{(i)}$ 和 $\boldsymbol{x}^{(j)}$ 不一定是相同的大小.  对于监督学习中每个样本的标签或目标, 我们可以使用一个数值码(numeric code)来表示(例如对于照片分类器, 0表示照片里面有人, 1表示有车, ...), 并且将这些数值码存在一个向量中, 例如 $\boldsymbol{y}$, 并且 $y_i$ 表示第 $i$ 个样本的标签, 不过有时候标签不止一个数值, 例如声音识别. 

**例子: 线性回归(linear regression)**

$\hat{y} = \boldsymbol{w}^\top \boldsymbol{x}$, 其中 $\boldsymbol{x} \in \mathbb{R}^n$ 为输入, $\boldsymbol{w} \in \mathbb{R}^n$ 称为 **参数(parameters)**, 我们可以把它理解为一个决定每一个特征 $x_i$ 影响预测值的能力的权重(weight)的集合, $y \in \mathbb{R}$ 为模型预测的值. 

所以在这里, T 就是用 $\hat{y} = \boldsymbol{w}^\top \boldsymbol{x}$ 来从输入 $\boldsymbol{x}$ 预测 $y$ . 对于性能测试 P, 我们可以用一个含有 $m$ 个输入样本的设计矩阵 $\boldsymbol{X}^{(\text{test})}$ 作为测试集(该测试集不能用来训练), 对于这些测试样本, 我们需要提前准确对应的正确的 $y$ 的向量$\boldsymbol{y}^{(\text{test})}$ . 一种度量性能的方式就是计算测试集的 **均方误差(mean squared error)**: $\text{MSE}_{\text{test}} = \frac{1}{m} \sum_i(\hat{\boldsymbol{y}}^{(\text{test}))} - \boldsymbol{y}^{(\text{test})})_i^2 = \frac{1}{m} \lVert \hat{\boldsymbol{y}}^{(\text{test})} - \boldsymbol{y}^{(\text{test})}\rVert_2^2$(越小越好, 也就是平方欧几里德距离除以 $m$ ).

为了构建一个机器学习算法, 我们需要算法能够通过观察训练集 $(\boldsymbol{X}^{(\text{train})}, \boldsymbol{y}^{(\text{train})})$ 获得经验来改善权重集 $\boldsymbol{w}$ 使得减小 $\text{MSE}_{\text{train}}$. 我们可以简单的找到 $\text{MSE}_{\text{train}}$ 关于 $\boldsymbol{w}$ 梯度为 0 的地方就是它的极值点, 

$$\nabla_{\boldsymbol{w}} \text{MSE}_{\text{train}} = 0$$
$$\Rightarrow \nabla_{\boldsymbol{w}} \frac{1}{m} \lVert \hat{\boldsymbol{y}}^{(\text{train})} - \boldsymbol{y}^{(\text{train})} \rVert_2^2 = 0$$
$$\Rightarrow \nabla_{\boldsymbol{w}} \frac{1}{m} \lVert \boldsymbol{X}^{(\text{train})} \boldsymbol{w} - \boldsymbol{y}^{(\text{train})} \rVert_2^2 = 0$$
$$\Rightarrow \nabla_{\boldsymbol{w}} (\boldsymbol{X}^{(\text{train})} \boldsymbol{w} - \boldsymbol{y}^{(\text{train})} )^\top (\boldsymbol{X}^{(\text{train})} \boldsymbol{w} - \boldsymbol{y}^{(\text{train})} ) = 0$$
$$\Rightarrow \boldsymbol{w} = \left( \boldsymbol{X}^{(\text{train})\top} \boldsymbol{X}^{(\text{train})}\right)^{-1} \boldsymbol{X}^{(\text{train})\top} \boldsymbol{y}^{(\text{train})} $$

通过上式给出的解的系统方程被称为 **正规方程(normal equations)**, 线性回归通过添加一个额外的 **偏差(偏置)参数(bais parameter)** $b$ 可以用来解决更加复杂一点的模型: $\hat{y} = \boldsymbol{w}^\top \boldsymbol{x} + b$(这样的一个映射叫做 **仿射函数(affine function)**, 本书后面提到仿射函数时会频繁地使用线性这个术语). 



### (p) 1.4.2 容量(Capacity), 过拟合(Overfitting) 和欠拟合(Underfitting)

机器学习算法最核心的挑战就是要在新的, 以前从未见过的数据集中表现良好, 这种能力被称为 **泛化(generalization)** . 在训练的时候, 我们希望降低 **训练误差(training error)**(也就是一个优化问题), 同样我们还希望 **泛化误差(generalization error)** 或 **测试误差(test error)** 也能很低, 泛化误差被定义为新输入的误差期望(期望的计算基于不同的可能的输入, 并且这些输入来自于系统在现实中遇到的分布).   一般的, 我们用测试集上的性能度量来估计泛化误差.

例如在线性回归上, 我们实际上需要关注的是测试误差: $\frac{1}{m} \lVert \boldsymbol{X}^{(\text{test})} \boldsymbol{w} - \boldsymbol{y}^{(\text{test})}\rVert_2^2$. 但是当我们只能看到训练集的时候怎么影响测试集上的性能? **统计学习理论(statistical learning theory)** 给出了一些答案. 我们必须知道训练集和测试集中的元素是怎么收集的, 也就是我们可以事先做出一些假设, 这样我们才可能去解决上述问题. 

训练集和测试集从数据集中按照概率分布生成这一过程称为 **数据生成过程(data-generating process)**, 我们作出以下假设(称为 **独立同分布假设(i.i.d. assumption))**: 训练集和测试集中的样本都是相互独立的, 训练集和测试集具有 **相同的分布**, 并且该分布称为 **数据生成分布(data-generating distribution)**, 记作 $p_{\text{data}}$ . 对于一个事先固定的 $\boldsymbol{w}$, 理论上训练误差和测试误差的期望是一样的, 而很多情况下, 我们是先通过选择对于训练集来说误差足够小的 $\boldsymbol{w}$, 再在测试集上测试, 所以在测试集上的测试误差肯定会大于等于训练误差, 所以性能良好的机器学习算法必须是使训练误差足够小, 并且还要使训练误差和测试误差的差距足够小. 

欠拟合就是训练误差太大, 过拟合就是训练误差和测试误差的差距太大. 一种控制模型过拟合和欠拟合行为的方法就是改变其 **容量(capacity)**, 通俗的说, 模型的容量就是模型所能表示各种不同函数的能力. 容量太小会使拟合训练集变得困难, 容量太大又会使模型记住太多训练集 **独有** 的一些属性. 一种改变模型的办法就是选择不同的 **假设空间(hypothesis space, 学习算法能够被允许产生的结果(函数)的集合(函数族))**, 例如对于线性规划, 算法的假设空间就是关于输入的所有线性函数, 我们可以通过把可能的结果设置为 **多项式(polynomials, 也就是添加 $x^i$ 作为新的线性规划的特征)** 而不是线性函数, 这样就能增加模型的容量. 例如 $\hat{y} = b + \sum_{i = 1}^{3} w_i x^i$ 就是一个三次方程, 如果把 $x^i$ 看作整体作为一个个特征(或则叫参数), 那么这样的一个方程仍然是线性函数.

![Figure 5.2][19]

> 上图从左至右分别是线性函数, 二次函数, 9 阶多项式拟合的结果(其中 9 阶多项式的可以完全拟合的结果有很多个, 这里只是其中的一种). 并且测试集是符合二次函数的点集. 可以看出, 左边欠拟合, 中间正好, 右边过拟合. 右边的 9 阶多项式使用 Moore-Penrose 伪逆去解那个正规方程(normal equation)得出的结果. 

容量不仅仅由模型的选择来决定, 模型规定了在调整参数训练目标时, 学习算法可以从哪些函数族中选择函数, 这被称为模型的 **表示容量(representational capacity)**, 但是往往找到最优解是困难的, 一般是找尽量减少训练误差的次优解, 加上这些限制条件(例如不要求找到最完美的优化解)之后, **有效容量(effective capacity)** 往往小于模型族的表示容量. 

提高机器学习模型泛化的现代思想可以依据 **简约思想(principle of parsimony)**: 在很多个都能解释观察现象的假设中, 我们应该选择最简单的那一个. (这个思想后面被统计学习理论的创始人形式化并准确化). 统计学习理论提供多种量化模型容量的方法, 其中最有名的就是 **Vapnik-Chervonenkis(VC) 维(demension)**, VC 维度量二元分类器(binary classify)的容量( **具体定义略** ). 

统计学习理论最重要的结论表明了训练误差和泛化误差之间的差距的上界随着模型容量的增长而增长并且随着训练样本的增多而减小. 但是这个解决很难用在深度学习中(而在机器学习中使用的很好), 因为这个界限经常很松(loose), 而且确定深度学习的模型容量相当困难(因为模型的有效容量受限于优化算法的能力, 并且对于一般的非凸(nonconvex)优化问题缺乏理论基础).

![Figure 5.3][20]

> 训练误差随着模型容量的上升而渐进与最小可能误差值(如果有的话, 一般很多实际应用中就会设定误差度量的最小值). 泛化误差以最优容量(optimal capacity)为最低点呈现 U-形.

为了处理任意高的容量的极端情况, 我们介绍 **非参数(nonparametric)**, 非参数模型没有参数模型的限制(在观察任意数据之前, 参数的个数就是有限且固定的). 有时候非参数模型仅仅是理论抽象的(也就是实际上做不出来), 不过我们可以设计一个函数复杂度与训练集大小正相关的函数作为非参数模型.

一个简单的非参数的例子就是 **最邻近回归(nearest neighbor regression)**, 最邻近回归直接简单的存储训练集中的 $\boldsymbol{X}$ 和 $\boldsymbol{y}$ , 然后在测试使用的时候, 就是简单的从训练集中找到一个与输入 $\boldsymbol{x}$ 距离(这个距离不仅仅是 $L^2$ 欧几里得距离, 还可以是例如 learned distance metrics)最近的一个点对应的 $y$ 返回. 显然, 在所有训练集上训练误差都是 $0$ 或 最小可能误差(如果对于训练集中相同输入不同输出的情况取平均的话).

我们还可以在一个参数会按照需求改变的算法中包裹一个参数学习算法来创建一个非参数学习算法. 

就算是理想模型(完全能够知道真实分布)都还是会遇到一些误差因为分布中的一些噪声. 例如在监督学习中, 有可能从 $\boldsymbol{x}$ 到 $y$ 的映射就是随机的或者是 $y$ 是一个还跟其他自变量关联的确定函数. 这种通过真实分布 $p(\boldsymbol{x}, y)$ 预测出现的误差被称为 **贝叶斯误差(Bayes error)** . 

**没有免费的午餐定理(No Free Lunch Theorem)**

考虑所有可能的数据生成分布并平均下来, 所有的分类算法在事先没有观察过的数据输入上都有相同的错误率. (因为就算是在某种分布下最优的模型, 在其他分布下就会得到更糟的结果, 差不多抵掉了).  也就是说没有适用于所有任意分布的"大一统"的最优学习算法.

但是现实生活中, 一般我们都会对可能的概率分布做一下假设(限制), 所以机器学习的主要目的不是研究出一个适用于所有情况(分布) 的学习算法, 而是仅仅关注于真实世界某一个特定问题(所以要给学习算法限定一个 **偏好(perference)** ), 找出一个近视的表现良好的分布.

**正则化(regularization)**

模型的效果取决于假设空间中的函数数量(也就是模型的表示容量)以及这些函数的具体形式.

对于线性回归, 我们可以添加对权重 $\boldsymbol{\omega}$ 的 **偏好**, 这种方法叫做 **权重衰减(weight decay)**  :

$$J(\boldsymbol{\omega}) = \text{MSE}_{\text{train}} + \lambda \boldsymbol{\omega}^\top \boldsymbol{\omega}$$

> 其中 $\lambda$(提前设定好的) 就是对权重的偏好, $\lambda = 0$ 表示对权重没有偏好, 对于取 $J(\boldsymbol{\omega})$ 最小值, $\lambda$ 越大表示偏好范数越小的权重(也就是权重中包含更少的特征).

跟一般的说, 正则化一个学习函数的模型时, 我们可以给代价函数(也就是上例中的 $J(\boldsymbol{\omega})$ ) 加上一个被称为 **正则化项(regularizer)** 的惩罚(penalty), 上例中的正则化项就是 $\boldsymbol{\omega}^\top \boldsymbol{\omega}$

![Figure 5.5][21]

> 上图是一个 9-维多项式回归的例子, $\lambda$ 就能用来控制过拟合和欠拟合(曲线的导数/梯度).

表达偏好要比直接删减假设空间中的函数更加一般地控制模型的容量, 例如删去一个成员函数相当于对这个函数表达无限大的一个偏好(使其权重边的无限小)

显式或隐式得对模型表达偏好的方法统称为 **正则化** .

> *Regularization is any modiﬁcation we make to a learning algorithm that is intended to reduce its generalization error but not its training error.*

就如没有免费的午餐定理所述, 同样一般来说也没有最优的正则化形式. 深度学习的哲学(尤其是本书中)就是能够通过一个非常通用的(general-purpose)正则化形式来解决大部分的任务.


  [1]: ./images/1516613842738.jpg
  [2]: ./images/1516613842738.jpg
  [3]: ./images/1516621710096.jpg
  [4]: ./images/1516624370367.jpg
  [5]: ./images/1516624435832.jpg
  [6]: ./images/1516624998426.jpg
  [7]: ./images/1516685167266.jpg
  [8]: ./images/1516697795762.jpg
  [9]: ./images/1517034158757.jpg
  [10]: ./images/1517036009925.jpg
  [11]: ./images/1517112955868.jpg
  [12]: ./images/1517134589413.jpg
  [13]: ./images/1517209835378.jpg
  [14]: ./images/1517211421206.jpg
  [15]: ./images/1517301723997.jpg
  [16]: ./images/1517302544553.jpg
  [17]: ./images/1517399058008.jpg
  [18]: ./images/1517580400888.jpg
  [19]: ./images/1518084054593.jpg
  [20]: ./images/1520217653005.jpg
  [21]: ./images/1520248674777.jpg
  [22]: ./images/1520316296099.jpg
  [23]: ./images/1520578241702.jpg


### (p) 1.4.3 超参数(Hyperparameters)和验证集(Validation Sets)

超参数不是由学习算法本身学习出来的(有时候这些设定很难优化, 所以一般是预先设定好的, 不过也可以做一个内嵌学习算法用来学习生成给外部学习算法的超参数).

大部分机器学习算法有 **超参数**, 例如前面说的多项式回归中有一个控制多项式次数的 **容量超参数(capacity hyperparameter)**, 还有上面的 $\lambda$ 超参数用于控制权重衰减的长度.

在训练过程优化超参数, 可以采用将训练集划分出 $20\%$ (典型地说) 作为 **验证集** (一个训练算法观察不到的样本数据集) , 另外 $80\%$ 还是跟原来一样的训练集用来学习出参数, 这个验证集用于在训练过程或训练之后估计泛化误差(因为验证集比较小, 所以通常验证集误差比训练误差小), 可以来更新超参数. 

> 实际中, 有时候测试集有点陈旧了, 而且里面包含的所有情况多差不多被学术界给研究透了, 我们就可能会得到一个看起来特别乐观的结果估计, 这不能反映真实性能. 反正要注意用更新更全面的测试集.

**交叉验证(cross-validation)**

如果验证集过小, 将会有问题, 因为这会造成在平均测试集误差的统计不确定性.

当样本过小的时候, 我们可以在原始数据上随机采样和分离出不同数据集来重复训练和测试. 最常见的手段是 **k-折(k-fold) 交叉验证**, 把原始数据分割成 $k$ 个不重合(nonoverlapping)的子集, 求 $k$ 次训练并测试的测试误差的平均作为最终的测试误差, 在 $i$ 次测试用, 将第 $i$ 个子集作为测试集, 其他的全部作为训练集.

> 上述 k-fold 方法带来的一个问题是不存在平均误差的方差的无偏估计(unbiased estimator), 但是通常用近似来解决. 

![Algo 5.1][22]

> $\boldsymbol{z}^{(i)}$ 在监督学习中是 $(\boldsymbol{x}^{(i)}, y^{(i)})$, 在无监督学习中是 $\boldsymbol{x}^{(i)}$

### (p) 1.4.4 估计(Estimators), 偏差(Bias) 和 方差(Variance)

**点估计(point estimation)**

点估计用来尝试找到一些感兴趣的量的单个最优预测, 这个感兴趣的量可以是参数模型中的一个参数或参数向量, 甚至是整个函数(这时候还叫做 **函数估计()** ).

> 为了区分参数的估计和它们的真正的值, 我们将参数 $\theta$ 的点估计记住 $\hat{\theta}$.
> 假设 $\theta$ 是固定且未知的, 而由于数据采样是随机的, 所以点估计 $\hat{\theta}$ 的值也是随机的, 也就是一个随机变量.

例如, 对于独立同分布(i.i.d)数据点集 $\boldsymbol{x}^{(1)}, \boldsymbol{x^{(m)}}$, 一个点估计或统计(statistic)可以是任意函数: $\boldsymbol{\hat{\theta}}_m = g(\boldsymbol{x}^{(1)}, \cdots, \boldsymbol{x}^{(m)})$. 并且没有要求点估计的值要和真实值接近(当然是越接近越好), 也没有要求函数的值域必须在真实值的合法范围, 这样能够让点估计的设计更加的灵活.

**偏差(Bias)**

一个估计的偏差定义为: 

$$\text{bias}(\hat{\theta}_m) = \mathbb{E}(\hat{\theta}_m) - \theta$$

当 $\text{bias}(\hat{\theta}_m) = 0$, 称为 **无偏(unbiased)**, 如果 $\lim_{m \rightarrow \infty} \text{bias}(\hat{\theta}_m) = 0$, 就叫做 **渐进无偏(asymptotically unbiased)** . 

**例子, 伯努利分布(Bernoulli Distribution)**

均值为 $\theta$ 的伯努利分布的独立同分布样本 $x^{(1)}, \ldots, x^{(m)}$.

$$P(x^{(i)}; \theta) = \theta^{x^{(i)}} (1 - \theta)^{1 - x^{(i)}}$$

一种常见的估计就是

$$\hat{\theta}_m = \frac{1}{m} \sum_{i=1}^{m} x^{(i)}$$

结合两式, 计算其偏差

$$
\begin{equation}
\begin{split}
\text{bias}(\hat{\theta}_m) &= \mathbb{E} (\hat{\theta}_m) - \theta \\
&= \frac{1}{m} \sum_{i=1}^{m} \mathbb{E} \left[ x^{(i)} \right] - \theta \\
&= \frac{1}{m} \sum_{i=1}^{m} \sum_{x^{(i)} = 0}^{1} x^{(i)} \theta^{x^{(i)}} (1 - \theta)^{1 - x^{(i)}} - \theta \\
&= \frac{1}{m} m \theta - \theta \\
&= 0
\end{split}
\nonumber
\end{equation}
$$


所以可以看出, 该估计是无偏差的.

同样的类似于上面过程的还可以证明估计高斯分布 $\mathcal{N} (x^{(i)}; \mu, \sigma^2)$的均值 $\mu$ 的估计的一般方法 $\hat{\mu}_m = \frac{1}{m} x^{(i)}$(样本均值) 是无偏的.

同样我们可以用样本方差来估计真实方差, 但是它是有偏差的, 证明如下

$$
\begin{equation}
\begin{split}
\text{bias}(\hat{\sigma}_m^2) &= \mathbb{E} [\hat{\sigma}_m^2] - \sigma^2 \\
&= \frac{1}{m} \mathbb{E}\left[ \sum_{i=1}^{m} \left( x^{(i)} - 2x^{(i)} \hat{\mu}_m + \hat{\mu}^2_m \right) \right] - \sigma^2 \\
&= \frac{1}{m} \mathbb{E} \left[ \sum_{i=1}^m \left( x^{(i)} \right)^2 - 2 \sum_{i=1}^m x^{(i)} \hat{\mu}_m + \sum_{i=1}^m \hat{\mu}_m^2 \right] - \sigma^2 \\
&= \frac{1}{m} \left( \mathbb{E} \left[ \sum_{i=1}^m \left( x^{(i)} \right)^2 \right] - \mathbb{E} \left( 2m \hat{\mu}_m^2 + m \hat{\mu}^2_m \right) \right) - \sigma^2 \\
&= \frac{1}{m} \sum_{i=1}^m \mathbb{E} \left[ (x^{(i)})^2 \right] - \mathbb{E} [ \hat{\mu}^2_m ] - \sigma^2 \\
&= \mathbb{E}^2 [ x ] + \mathtt{Var} [ x ] - \mathbb{E}^2 [ \hat{\mu}_m ] - \mathtt{Var}[\hat{\mu}_m] - \sigma^2 \\
&= \sigma^2 - \mathtt{Var}[\hat{\mu}_m] - \sigma^2 \\
&= - \mathtt{Var} \left[ \frac{1}{m} \sum_{i=1}^m x^{(i)} \right] \\
&= - \frac{1}{m} \sum_{i=1}^m \mathtt{Var} [ x^{(i)}] \\
&= - \frac{1}{m} \sigma^2
\end{split}
\nonumber
\end{equation}
$$

因为偏差不为 $0$, 所以这个估计是有偏差的, 修正为无偏差估计只需要换一下这个归一公式的系数

$$\tilde{\sigma}^2_m = \frac{1}{m-1} \sum_{i=1}^m \left( x^{(i)} - \hat{\mu}_m \right)^2$$

**方差(Variance)** 和 **标准差(standard error)**

类似于计算估计量的数学期望, 我们还会计算其方差 $\mathtt{Var} (\hat{\theta})$, 以及标准差(方差的平方根) $\text{SE}(\hat{\theta})$.

因为估计在统计上会随着训练样本的变化而发生变化, 所以方差衡量这些估计离真实值的变化程度的大小.

不过不管是用样本方差的平方根还是方差的无偏估计(也就是上面高斯分布的那个修正的方差估计)的平方根来估计标准差都是有偏的(都倾向于低估(underestimate)真实方差), 不过方差的无偏估计的平方根低估的少一点, 对于数量很大的样本, 还是有意义的.

例如这里用样本均值的方差的平方根估计其标准差, 由前面高斯分布的方差的有偏估计的推导, 很容易得出

$$
\text{SE} (\hat{\mu}_m) = \sqrt{\mathtt{Var} \left[ \frac{1}{m} \sum_{i=1}^{m} x^{(i)} \right]} = \frac{\sigma}{\sqrt{m}}
$$

又由 **中心极限定理(central limit theorem)**, 这些服从正态分布的随机变量的均值也一样服从正态分布. 均值的标准差在机器学习实验中很有用, 例如均值为 $\hat{\mu}_m$, 方差 $\text{SE}(\hat{\mu}_m)^2$ 的高斯分布的以均值 $\hat{\mu}_m$ 为中心 $95\%$(也就是该高斯函数在该置信区间上的积分是整个定义域上积分的值的 $95\%$) 的 **置信区间(confidence interval)** 为 $\left( \hat{\mu}_m - 1.96 \text{SE}(\hat{\mu}_m), \hat{\mu}_m + 1.96 \text{SE}(\hat{\mu}_m) \right)$  (区间估计)
  
  算法 A 比算法  B 好通常是指算法 A 误差的 $95\%$ 置信区间的上界都比算法 B 的 $95\%$ 置信区间的下界还小.
  
  **例子: 伯努利分布**
  
  同样类似上面的例子, 我们可以求得均值的方差

$$
  \begin{equation}
  \begin{split}
  \mathtt{Var}(\hat{\theta}_m) &= \mathtt{Var} \left( \frac{1}{m} \sum_{i=1}^m x^{(i)} \right) \\
  &= \frac{1}{m^2} \sum_{i=1}^m \mathtt{Var}(x^{(i)}) \\
  &= \frac{1}{m^2} \sum_{i=1}^m \left( \mathbb{E}\left[ \left( x^{(i)} \right)^2 \right] - \mathbb{E}^2 [x^{(i)}] \right) \\
  &= \frac{1}{m^2} \sum_{i=1}^m \left( 1^2 \Pr (x^{(i)} = 1) + 0 - \left( 1 \Pr (x^{(i)} = 1) + 0 \right)^2 \right) \\
  &= \frac{1}{m} \theta (1 - \theta)
  \end{split}
  \nonumber
  \end{equation}
$$
  
  **权衡(trading off)方差和偏差来最小化MSE**
  
  如果有两种估计方法, 一个估计方法的偏差很大而另外一个的方差很大的情况下, 我们可以用均方误差(MSE)作为惩罚的交叉验证来权衡方差和偏差:
  
  $$
\begin{eqnarray} 
  \text{MSE} &= \mathbb{E}\left[ \left( \hat{\theta}_m - \theta \right)^2 \right] \\
  &=& \mathbb{E} \left[ (\hat{\theta}_m)^2 - 2 \hat{\theta}_m \theta + \theta^2 \right] \\
  &=& \mathbb{E} \left[ \left( \hat{\theta}_m \right) \right] - 2 \theta \mathbb{E}[\hat{\theta}_m] + \theta^2 \\
  &=& \left( \mathbb{E} [ \hat{\theta}_m^2] - \theta \right)^2 - \mathbb{E}^2 [\hat{\theta}_m] + \mathbb{E} \left[ \left( \hat{\theta}_m \right)^2 \right] \\
  &=& \text{bias}^2 \hat{\theta}_m + \mathtt{Var} (\hat{\theta}_m)
\end{eqnarray} 
  $$

> [Reference: Wiki](https://en.wikipedia.org/wiki/Mean_squared_error)

![Figure 5.6][23]

偏差和方差与学习算法的模型的容量有一定关系, 随着模型的容量的增加, 方差和偏差分别有增加和较小的趋势.

**一致性(consistency)**

前面我们说的都是在固定大小训练集上的各种估计的属性, 我们还关注训练集大小 $m$ 不断增加时点估计的性质. 一般我们想要 **一致性(有时候也被称为弱一致性 weak consistency)** :

$$\text{plim}_{m \rightarrow \infty} \hat{\theta}_m = \theta$$

> 其中 $\theta$ 为要估计的参数, $\text{plim}$ 表示 **依概率收敛(convergence)** , 也就是对于任意 $\epsilon > 0$, 当 $m \rightarrow \infty$, 有 $P(|\hat{\theta}_m - \theta| > \epsilon) \rightarrow 0$.

相对于上面的弱一致性, **强一致性(strong consistency)** 是指 **几乎必然(almost sure)** 从 $\hat{\theta}$ 收敛到 $\theta$, 随机变量 $\boldsymbol{\mathtt{x}}^{(1)}, \ldots$几乎必然收敛到 $x$ 是指 $p(\lim_{m \rightarrow \infty} \boldsymbol{\mathtt{x}}^{(m)} = x) = 1$.

**总之, 按照书上的结论, 一致(consistent)表明该估计的偏差一定会随着样本数量的增大而减小.**

例如, 我们现在要估计的量 $\theta$ 的真实值为 $1$, 样本集为 $x^{(i)} \text{ drawn from } \{-1, 1\}$ 并且它们是 **i.i.d** 并且服从 $p = 0.5$ 的伯努利分布, 此时我们有一种估计方法 $\hat{\theta} = x^{(1)} + 1$, 很明显我们可以得出其偏差为 $0$ (因为其数学期望为 $1$), 但是这个数学期望相对于真实值的偏差并 **不会** 随着样本个数的增加而减少(因为一直是 $0$), 或者形式化得说, $\text{for any } \epsilon > 0, \Pr(|\hat{\theta} - \theta| > \epsilon) \equiv 1$ 而不会因为 $m \rightarrow \infty$ 而趋近于 $0$ (这里面直接把 $\hat{\theta} = x^{(1)} + 1$ 带进去, 也就是 $\hat{\theta}$ 要么是 $0$ 要么是 $2$ ).

> 上面例子 [参考](https://www.quora.com/Are-unbiased-estimators-always-consistent)


  [1]: ./images/1516613842738.jpg
  [2]: ./images/1516613842738.jpg
  [3]: ./images/1516621710096.jpg
  [4]: ./images/1516624370367.jpg
  [5]: ./images/1516624435832.jpg
  [6]: ./images/1516624998426.jpg
  [7]: ./images/1516685167266.jpg
  [8]: ./images/1516697795762.jpg
  [9]: ./images/1517034158757.jpg
  [10]: ./images/1517036009925.jpg
  [11]: ./images/1517112955868.jpg
  [12]: ./images/1517134589413.jpg
  [13]: ./images/1517209835378.jpg
  [14]: ./images/1517211421206.jpg
  [15]: ./images/1517301723997.jpg
  [16]: ./images/1517302544553.jpg
  [17]: ./images/1517399058008.jpg
  [18]: ./images/1517580400888.jpg
  [19]: ./images/1518084054593.jpg
  [20]: ./images/1520217653005.jpg
  [21]: ./images/1520248674777.jpg
  [22]: ./images/1520316296099.jpg
  [23]: ./images/1520578241702.jpg

### (p) 1.4.5 最大似然估计(Maximum Likelihood Estimation)

我们需要对不同模型使用不用的优秀的估计函数提供原理, 而不是去瞎猜. 其中最常用的原理就是 **最大似然估计**.  

假设 $m$ 个样本的集合 $\mathbb{X} = \{ x^{(1)}, \ldots, x^{(m)} \}$ 独立地由真实但是未知的概率分布 $p_{\text{data}}(\mathtt{x})$ 生成. 让 $p_{\text{model}}(\mathtt{x}; \boldsymbol{\theta})$ 为由 $\boldsymbol{\theta}$ ( $\boldsymbol{\theta}$ 为模型的(向量)参数 )确定的在相同空间上的一参数族(parametric family, 表示为 $\{ p_{\text{model}} (\mathtt{x}; \boldsymbol{\theta}) | \boldsymbol{\theta} \in \boldsymbol{\Theta} \}$ )的概率分布.

最大似然估计找到这个参数 $\boldsymbol{\theta}$ 的最大似然估计 $\boldsymbol{\theta}_{\text{ML}}$ 使得最大化 **似然函数(Likelihood function)** $\mathcal{L} (\boldsymbol{\theta}; \mathtt{x})$.

在本例中

$$\boldsymbol{\theta}_{\text{ML}} = \text{arg} \max_{\boldsymbol{\theta}} p_{\text{model}}(\mathbb{X}; \boldsymbol{\theta}) = \text{arg} \max_{\boldsymbol{\theta}} \prod_{i=1}^m p_{\text{model}} (x^{(i)}; \boldsymbol{\theta})$$

> 根据 i.i.d 假定每个样本之间都是独立事件.

但是上面的乘积有一点不方便, 例如可能会造成数字下溢, 为了方便, 我们经常使用对数似然(log-likelihood)来等价的替代上述优化问题, $\boldsymbol{\theta}_{\text{ML}} = \text{arg} \max_{\boldsymbol{\theta}} \sum_{i=1}^m \log p_{\text{model}}(x^{(i)}; \boldsymbol{\theta})$.  

**MLE 和 cross-entropy 的关系**

**empirical distribution** 就是简单的把样本 $X_1,X_2,\cdots,X_n$ 看成一个随机变量的所有取值，且取每个值的概率为 $\frac{1}{n}$

$$\mathbb{E}_{\mathtt{x} \sim f_{\text{emprical}}(\mathtt{x})} [ x ] = \frac{1}{n}\sum_i x_i$$

根据 Glivenko-Cantelli theorem

${\displaystyle \|F_{n}-F\|_{\infty }=\sup _{x\in \mathbb {R} }|F_{n}(x)-F(x)|\longrightarrow 0}\ a.s.$

随着样本 $n$ 的增加经验分布 $F_n$ 一直逼近真实分布 $F$ 。因此 $F_n$ 可以看作是 $F$ 的近似。

所以我们为了使得 $p_{\text{model}}$ 尽量逼近 $p_{\text{real}}$ ( true but unknown distri. ), 但是没法知道 $p_{\text{real}}$, 我们就尽量逼近 $p_{\text{empirical}}$

所以问题转为 $\boldsymbol{\theta}_{\text{ML}} = \text{arg} \max_{\boldsymbol{\theta}} \frac{1}{m} \sum_{i=1}^m \log p_{\text{model}}(x^{(i)}; \boldsymbol{\theta}) = \arg \max_{\boldsymbol{\theta}} \mathbb{E}_{\mathtt{x} \sim p_{\text{empirical}}} \log p_{\text{model}}(x^{(i)}; \boldsymbol{\theta}) = \arg \max_{\boldsymbol{\theta}} \left( - H (p_{\text{empirical}}, p_{\text{model}}) \right)$

> 因为 $D_{\text{KL}} (P || Q) = \mathbb{E}_{\mathtt{x} \sim P(\mathtt{x})} [ \log P(x) - \log Q(x) ] = - H(P) + H(P, Q)$, 而对于上面 $\arg \max_\theta$, 自信息 $H(p_{\text{empirical}})$ 不含 $\theta$ 项目, 所以只剩下交叉熵 $H(P, Q)$. 

### (p) 1.4.6 贝叶斯统计(Bayesian Statistics)

前面讨论的都是 **频率派统计(frequentist statistics, 是对固定但是未知的参数 $\theta$ 的单一值估计 $\hat{\theta}$, 而这个点估计是一个随机变量)**, 贝叶斯统计则会考虑要估计的参数 $\theta$(贝叶斯派认为这个参数是一个随机变量) 的所有可能的值.	

我们将对未知参数 $\boldsymbol{\theta}$(随机变量) **已知的知识**(在观测数据集之前的) 作为 **先验概率分布(prior probability distribution) $p(\boldsymbol{\theta})$**,  一般这个先验的选取都是很 **宽泛(墒值很高)** 	的分布, 比如在无限大面积或体积的均匀分布(也就是各种可能的点都是相同概率, 并且在整个可行域上都有分布)或者高斯分布, 然后得到的 **后验(posterior)概率分布** 一般是 **低墒且概率集中在几个最有可能的参数值上面**.

> e.g. 对于期末考试学生成绩的预测, 平时成绩就可以作为要预测的那个学生的先验.

在观察 $m$ 个样本后, 预测第 $m + 1$ 个样本:

$$p(x^{(m+1)} | x^{(1)}, \cdots, x^{(m)}) = \int p(x^{(m+1)}, \boldsymbol{\theta} | x^{1}, \cdots, x^{(m)}) d\boldsymbol{\theta} = \int p(x^{(m + 1)} | \boldsymbol{\theta}, x^{(1)}, \cdots, x^{(m)}) p(\boldsymbol{\theta} | x^{(1)}, \cdots, x^{m}) = \int p(x^{(m + 1)} | \boldsymbol{\theta}) p(\boldsymbol{\theta} | x^{(1)}, \cdots, x^{(m)}) d \boldsymbol{\theta}$$

> 其中 $p(x^{(m+1)} | \boldsymbol{\theta}, x^{(1)}, \cdots, x^{(m)}) = p(x^{(m+1)} | \boldsymbol{\theta})$ 是因为 $x^{(m+1)}$ 和 $x^{(1)}, \cdots, x^{(m)}$ 是 **互斥** 的.


每个正概率密度的 $\boldsymbol{\theta}$ 都为预测下一个(第 m + 1 个)样本做贡献, 并且乘上去的都是 $\boldsymbol{\theta}$ 的后验概率 $p(\boldsymbol{\theta} | x^{(1)}, \cdots, x^{(m)})$. 

如果是频率派的观点, $\boldsymbol{\theta}$ 的点估计是有不确定度(uncertainty)的, 并且用其点估计的方差来做区间估计, 而贝叶斯派则是直接考虑了所有 $\boldsymbol{\theta}$ 的可能值的后验概率并进行积分(**往往可以防止过拟合**), 后验概率分布的墒也就反映了其不确定度.

贝叶斯方法还有一个特点是先验的选择有 **人为主观因素** 在里面.

在训练样本有限的时候, 贝叶斯方法通常泛化得更好, 而数据规模大的时候计算量太大了. 

**贝叶斯线性回归**

贝叶斯方法不用像前面提到的线性回归那样, 通过正则化来添加一个对模型复杂度的偏好 $\lambda$ 来控制模型的复杂度, 而可以直接避免像最大似然估计那样会导致模型过拟合的情况.

$$\hat{y} = \boldsymbol{w}^\top \boldsymbol{x}, \boldsymbol{x} \in \mathbb{R}^n, y \in \mathbb{R}, \boldsymbol{w} \in \mathbb{R}^n$$
$$\hat{\boldsymbol{w}}^{(\text{train})} = \boldsymbol{X}^{(\text{train})} \boldsymbol{w}$$

用 $\boldsymbol{y}^{(\text{train})}$ 上的高斯条件分布(假设该条件概率服从高斯分布, 因为往往我们可以认为 $y = \hat{y} + \epsilon$, 其中噪声 $\epsilon$ 为服从均值为 0 的高斯分布的随机变量)可是表示为(省略了 train 上标):

$$p(\boldsymbol{y} | \boldsymbol{X}, \boldsymbol{w}) = \mathcal{N}(\boldsymbol{y}; \boldsymbol{Xw}, \boldsymbol{I}) \propto \exp\left(- 0.5(\boldsymbol{y} - \boldsymbol{X w})^\top(\boldsymbol{y} - \boldsymbol{X w}) \right)$$


![Gaussian conditional distribution for t given by x](./images/1531366660088.png)

> t 就是相当于前面公式的 y

又设 $\boldsymbol{w}$ 的先验为高斯分布 $p(\boldsymbol{w}) = \mathcal{N}(\boldsymbol{w}; \boldsymbol{\mu}_0, \boldsymbol{\Lambda}_0) \propto \exp\left( -0.5(\boldsymbol{w} - \boldsymbol{\mu}_0)^\top \boldsymbol{\Lambda}_0^{-1} (\boldsymbol{w} - \boldsymbol{\mu}_0) \right)$ (而且一般 $\boldsymbol{\mu}_0 = 0, \boldsymbol{\Lambda}_0 = \frac{1}{\alpha} \boldsymbol{I}$, $\alpha$ 是方差的倒数, 在贝叶斯方法中常被称为 **精度**, 这个时候贝叶斯方法估计出来的 $\boldsymbol{w}$ 跟前面说到的频率派的带有 $\alpha \boldsymbol{w}^\top\boldsymbol{w}$ 容量限制惩罚的正则项的线性回归 )

所以 $\boldsymbol{w}$ 的后验概率为

$$p(\boldsymbol{w} | \boldsymbol{X}, \boldsymbol{y}) \propto p(\boldsymbol{y} | \boldsymbol{X}, \boldsymbol{w}) p(\boldsymbol{w}) \propto \exp\left(-0.5(\boldsymbol{w} - \boldsymbol{\mu}_m)^\top \boldsymbol{\Lambda}_m^{-1}(\boldsymbol{w} - \boldsymbol{\mu}_m) \right)$$

其中 $\boldsymbol{\Lambda}_m = \left(\boldsymbol{X}^\top\boldsymbol{X} + \boldsymbol{\Lambda}_0^{-1}\right)^{-1}, \boldsymbol{\mu}_m = \boldsymbol{\Lambda}_m(\boldsymbol{X}^\top\boldsymbol{y} + \boldsymbol{\Lambda}_0^{-1}\boldsymbol{\mu}_0)$

> 上面的推导过程中省略所有不含 $\boldsymbol{w}$ 的项, 因为那些项在条件概率的上下文下都是已知的, 不会对 $\boldsymbol{w}$ 的后验分布造成影响. 这个推导里面最难的就是用 $\boldsymbol{\Lambda}_m$ 和 $\boldsymbol{\mu}_m$ 凑成高斯分布.

**最大后验估计(Maximum Posterior Estimation, abbr., MAP)**

前面提到的完整贝叶斯后验分布的计算量比较大, 有时候我们依然点估计使用较少的计算量来提供一个可行的近似解, 我们依然可以加入先验来影响点估计, 这就称为 **最大后验点估计**.

MAP 选择后验概率最大的点:

$$\theta_{\text{MAP}} = \arg \min_\theta p(\theta | x) = \arg \min_\theta \left(\log p(x | \theta) + \log p(\theta)\right)$$

前面带有正则化的最大似然估计(带有 $\lambda w^\top w$ 惩罚的就是正比于对数先验项)就可以看作是贝叶斯推断的MAP近似.

MAP 提供了一个直观的方法来设计复杂但是可以解释的正则化项.